In [ ]:
a state of trance 274

In [ ]:
# Process with custom output directory
parsed_data, md_file, json_file = process_asot_episode(
    "https://www.astateoftrance.com/episode-313/", 
    output_dir="asot"
)

In [14]:
from process_asot_episode import process_asot_episode, extract_episode_number
from unity_json import read_and_merge_json_files

In [16]:
episodes = [
'https://www.astateoftrance.com/episode-274/',
'https://www.astateoftrance.com/episode-313/',
'https://www.astateoftrance.com/episode-314/',
'https://www.astateoftrance.com/episode-325/',
'https://www.astateoftrance.com/episode-331/',
'https://www.astateoftrance.com/listen-now-asot1119/'
]

existing_episodes = milvus_client.list_episodes("asot")

requested_episodes =  [episode for episode in episodes if extract_episode_number(episode) not in existing_episodes]
requested_episodes

2025-04-13 21:29:41,762 - DEBUG - milvus_logger - Found 6 unique episodes in collection asot


['https://www.astateoftrance.com/listen-now-asot1119/https://www.astateoftrance.com/episode-339/']

In [ ]:


episodes = [
'https://www.astateoftrance.com/episode-274/',
'https://www.astateoftrance.com/episode-313/',
'https://www.astateoftrance.com/episode-314/',
'https://www.astateoftrance.com/episode-325/',
'https://www.astateoftrance.com/episode-331/',
'https://www.astateoftrance.com/listen-now-asot1119/'
]

data_dir = "data/asot"
results = []
for url in episodes:
    try:
        result = process_asot_episode(url, output_dir=data_dir)
        results.append(result)
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

all_records = read_and_merge_json_files(data_dir) # ready to be loaded into milvus


Processing ASOT Episode 274 from URL: https://www.astateoftrance.com/episode-274/
Step 1: Episode number extracted.
Step 2: Starting web scraping...
Scraping https://www.astateoftrance.com/episode-274/ (Attempt 1/3)
Step 2: Web scraping completed.
Step 3: Saving raw markdown content...
Step 3: Saved raw markdown to data/asot/asot_episode_274_raw.md
Step 4: Parsing song list using Claude...
Step 4: Song list parsing completed.
Step 5: Saving parsed data to JSON...
Step 5: Saved parsed song data to data/asot/asot_episode_274.json
Successfully processed ASOT Episode 274.
Found 6 JSON files in data/asot
Processed 128 total records


In [2]:
  # Directory containing JSON files
collection_name = "asot"  # Name of the Milvus collection

In [3]:
from MilvusClientASOT import MilvusClientASOT
milvus_client = MilvusClientASOT()
milvus_client.client.list_collections()

/Users/tirsolopezausens/Desktop/aisot/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-04-13 21:19:25,967 - DEBUG - milvus_logger - Milvus Client successfully initialized.
2025-04-13 21:19:29,238 - DEBUG - milvus_logger - Dense embeddings initialized.


['asot']

In [6]:
milvus_client.insert_new_episodes(collection_name, all_records)

2025-04-13 21:21:41,195 - DEBUG - milvus_logger - Found 5 unique episodes in collection asot
2025-04-13 21:21:41,196 - DEBUG - milvus_logger - Found 5 existing episode IDs in asot.
2025-04-13 21:21:41,196 - INFO - milvus_logger - Skipped 107 documents because their episode_id already exists or was missing.
2025-04-13 21:21:42,389 - DEBUG - milvus_logger - Prepared 21 documents with embeddings
2025-04-13 21:21:42,404 - DEBUG - milvus_logger - Inserted 21 documents into asot
2025-04-13 21:21:42,405 - INFO - milvus_logger - Successfully inserted 21 new episode documents into asot.


{'insert_count': 21, 'ids': [457326097369927986, 457326097369927987, 457326097369927988, 457326097369927989, 457326097369927990, 457326097369927991, 457326097369927992, 457326097369927993, 457326097369927994, 457326097369927995, 457326097369927996, 457326097369927997, 457326097369927998, 457326097369927999, 457326097369928000, 457326097369928001, 457326097369928002, 457326097369928003, 457326097369928004, 457326097369928005, 457326097369928006], 'cost': 0}

In [6]:
milvus_client.client.drop_collection("asot")

In [7]:
milvus_client.list_episodes("asot")

2025-04-13 21:21:53,763 - DEBUG - milvus_logger - Found 6 unique episodes in collection asot


['1119', '274', '313', '314', '325', '331']

In [ ]:
result = milvus_client.create_and_load_collection(collection_name, all_records)

In [10]:
# You can play with Milvus vector database here to check job positions
# This job positions will be retrieved by the RAG agent
test_query = """
im alone 274
"""

print("=" * 50)
print(f"\nQuerying with skills: {test_query}")
print("=" * 50)

results = milvus_client.sparse_search(
    collection_name=collection_name,
    query_text=test_query,
    limit=10,
)
print(f"Found {len(results)} matches:")
for i, hit in enumerate(results):
    print(f" {i}. Episode: {hit['entity']['episode_id']}")
    if hit['entity']['artist'] != 'nav':
        print(f" Artist: {hit['entity']['artist']}")
    if hit['entity']['collaborators'] != 'nav':
        print(f" Collaborators: {hit['entity']['collaborators']}")
    if hit['entity']['featured_artists'] != 'nav':
        print(f" Featured Artists: {hit['entity']['featured_artists']}")
    if hit['entity']['title'] != 'nav':
        print(f" Title: {hit['entity']['title']}")
    if hit['entity']['remix_info'] != 'nav':
        print(f" Remix: {hit['entity']['remix_info']}")
    if hit['entity']['ranking'] != -1:
        print(f" Ranking: {hit['entity']['ranking']}")
    if hit['entity']['popularity_score'] != -1:
        print(f" Popularity Score: {hit['entity']['popularity_score']}")
    if hit['entity']['vote_count'] != -1:
        print(f" Vote Count: {hit['entity']['vote_count']}")
    if hit['entity']['URL'] != 'nav':
        print(f" URL: {hit['entity']['URL']}")
    print(f" Match Score: {hit['distance']:.4f}")



Querying with skills: 
im alone 274

Found 10 matches:
 0. Episode: 274
 Artist: Sun Decade
 Title: I'm Alone
 Remix: (Stoneface & Terminal Remix)
 Ranking: 10
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 6.8160
 1. Episode: 274
 Artist: Aurosonic
 Title: Starfall
 Ranking: 2
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 2.7320
 2. Episode: 274
 Artist: Bolier & Coenraad
 Title: Mohawk
 Ranking: 19
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 2.6777
 3. Episode: 274
 Artist: Aly & Fila
 Title: Ankh
 Ranking: 15
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 2.6777
 4. Episode: 274
 Artist: Robert Nickson
 Title: untitled
 Ranking: 14
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 2.6777
 5. Episode: 274
 Artist: Carl B
 Title: Chasing Leaves
 Ranking: 13
 URL: https://www.astateoftrance.com/episode-274/
 Match Score: 2.6255
 6. Episode: 274
 Artist: Joint Operations Centre
 Title: Shortwave
 Rank

In [ ]:

# Import the MilvusClientASOT class
from milvus_client import MilvusClientASOT  # Assuming this is the file name containing your class


"""Main function to process ASOT data and insert into Milvus."""
# Configuration
data_dir = "data/asot"  # Directory containing JSON files
collection_name = "asot_songs"  # Name of the Milvus collection

# Initialize Milvus client
milvus_client = MilvusClientASOT()

# Find, read and merge all JSON files
all_records = read_and_merge_json_files(data_dir)

# Check if collection exists and delete it if needed
if milvus_client.client.has_collection(collection_name):
    print(f"Collection {collection_name} already exists. Deleting it first...")
    milvus_client.delete_collection(collection_name)

# Use the client's method to create and load the collection
result = milvus_client.create_and_load_collection(collection_name, all_records)
print(f"Successfully inserted data into {collection_name}")


In [1]:
# found formats for now for URLS
https://www.astateoftrance.com/episode-313/
https://www.astateoftrance.com/a-state-of-trance-episode-870/
https://www.astateoftrance.com/asot-1110/
https://www.astateoftrance.com/listen-now-asot1119/

In [ ]:
songs_raw = scrape_url_with_retry('https://www.astateoftrance.com/episode-313/')

In [18]:
from song_parser import parse_songs_with_claude
import json

episode_id = '313'
songs_extracted = parse_songs_with_claude(songs_raw)
json_file_name = f'parsed_songs_asot_{episode_id}.json'

import json
with open(json_file_name, 'w') as f:
    json.dump(songs_extracted, f)

In [ ]:
import os
import json
from MilvusClientASOT import MilvusClientASOT
milvus_client = MilvusClientASOT()

In [ ]:
milvus_client.client.list_collections()

In [ ]:
# Create a collection with the job positions if it doesn't exist
# Note you can do this also by running the job_positions_ingestion.py script
# This is just for free-play

collection_name = os.getenv("MILVUS_COLLECTION")

if milvus_client.client.has_collection(collection_name):
    print(f"Collection {collection_name} already exists. Skipping.")
else:
    print(f"Collection {collection_name} does not exist. Creating...")
    job_positions_file_path = "job_positions.json"
    with open(job_positions_file_path, 'r') as f:
        job_positions = json.load(f)
    result = milvus_client.create_and_load_collection(collection_name, job_positions)


In [ ]:
# You can play with Milvus vector database here to check job positions
# This job positions will be retrieved by the RAG agent
test_query = """
light the skies retrobyte
"""

print("=" * 50)
print(f"\nQuerying with skills: {test_query}")
print("=" * 50)

# results = milvus_client.hybrid_search(
#     collection_name=collection_name,
#     query_text=test_query,
#     limit=3,
#     ranker_type="weighted",
#     sparse_weight=0.9,
#     dense_weight=0.1,
#     output_fields=["episode_id", "text"]
# )

results = milvus_client.sparse_search(
    collection_name=collection_name,
    query_text=test_query,
    limit=3,
)

print(f"Found {len(results)} matches:")
for i, hit in enumerate(results):
    print(f"  {i}. {hit['entity']['episode_id']}")
    print(f"     Match Score: {hit['distance']:.4f}")
    print(f"     Required Skills: {hit['entity']['text']}")
    print()

In [5]:
milvus_client.client.drop_collection(collection_name)

In [18]:
asot314 = """
1. Jody Wisternoff – Starstrings (high contrast recordings)
2. Bobina – The Last He Said
3. Nic Chagall – Borderline (high contrast recordings)
4. Unknown Artist – Interference
5. Signalrunners – Dont look back (Anjunabeats)
6. Oakenfold – Not Over (Robert Vadneys 2007 remix) (Perfecto)
7. Terry Ferminal – Deep Inside (high contrast recordings)
8. Mark Norman – Blikkenmachine (Blackhole)
9. The Doppler Effect – Beauty Hides in the Deep (John O Callaghan
remix)(Armada)
10. Andy Bagguley – Together As one (Michael Dow remix) (Instant Music)
11. FUTURE FAVORITE: Envio – For You (The Blizzard remix) (ASOT)
12. TUNE OF THE WEEK: Tom Colontonio – Oxidize (Conspiracy Recordings)
13. Bart Claessen – First Light (United)
14. Paul van Dyk – Castaway (Positiva)
15. Solarstone – 7 Cities (Tom Colontonio Rework)
16. Joop – Prominent (high contrast recordings)
17. Armin van Buuren – The Sound Of Goodbye (Nic Chagalls drumbeat re-edit)
18. Digital Nature – Oasis (John Askew remix) (Flux Delux)
19. Above & Beyond – Home (Above & Beyond club mix)
20. ASOT Radio Classic: Solid Sessions – Janeiro (Armin van Buuren remix)
(Positiva)
"""

asot325 = """1. Armin van Buuren – The Sound of Goodbye (EDXs Indian Summer remix)(Sirup/Armada)
2. Fragma – Toca Me (Inpetto 2008 remix)
3. Myon – Albion
4. Milan Lieskovsky – Elenya (Heatbeat remix) (Bonzai)
5. Arnej – Rendezvous (Armada)
6. First State – Sierra Nevada (from the album Time Frame) (Black Hole)
7. Topher Jones – Young Life
8. Mystery Islands – Perfect Escape (Octagen remix) (Insight recordings)
9. Markus Schulz feat. Departure – Cause You Know (Nic Chagall remix) (Coldharbour Recordings)
10. Alan M – Famicom (Temple One remix)
11. Deep Voices – Your Picture (Robert Nickson remix) (Combined Forces)
12. Luke Warner & Mat Lock – Deep Psychosis (Daniel Kandis Cure remix)
13. Alex Kunnari – Lifter (Paul Webster remix) (Black Hole)
14. Digital Nature & Onova – Inspiration
15. TUNE OF THE WEEK: Aly & Fila – Lost Language (Armada/ Off Shore)
16. The Doppler Effect – Beauty hides in the Deep (John O Callaghan remix)(Armada)
17. FUTURE FAVORITE: Andy Blueman – Nyctalopia (Onova remix) (Perceptive Recordings)
18. Cliff Coenraad – Manjula (Intuition Recordings)
19. ASOT Radio Classic: Push – Strange World 2000 remake (Bonzai)"""

asot331 = """
Top 20 of 2007 as voted by listeners to ASOT
Number of confirmed valid votes: 11340

Statement:
This list is a true and accurate reflection of all votes and purely based on listeners votes. Each listener that submitted his or her votes, selected his personal top 5 of 2007 of tracks, as played in one of the shows in 2007. All votes were checked on possible fraud. Armin nor Armada had anything to do with the points awarded.

Special thanks to Peter Kruit (www.arminvanbuuren.net)
Position / Track / Points / Votes
1. John O Callaghan feat. Audrey Gallagher – Big Sky (Agnelli & Nelson Remix) 4703 1415
2- Armin van Buuren vs Rank 1 feat. Kush – This World Is Watching Me (Cosmic Gate Remix) 3414 948
3. Dash Berlin – Till The Sky Falls Down (Vocal Mix) 2607 821
4. Above & Beyond – Home (Above & Beyond Club Mix) 2578 850
5. The Blizzard – Kalopsia (Original Mix) 2552 780
6. Armin van Buuren – Rush Hour 2205 726
7. First State feat. Anita Kelsey – Falling 1977 682
8. Vincent de Moor – Fly Away (Cosmic Gate Remix) 1969 623
9. Paul van Dyk & Starkillers & Austin Leeds featuring Ashley Tomberlin – New York City 1901 648
10. Armin van Buuren – The Sound Of Goodbye (Nic Chagalls Drumbeat Re-Edit) 1807 582
11. Filo & Peri featuring Eric Lumiere – Anthem (Original Mix) 1724 533
12. Above & Beyond present Tranquility Base – Oceanic (Original Mix) 1714 559
13. Ilse De Lange – The Great Escape (Armin van Buuren Remix) 1687 556
14. Jonas Steur featuring Jennifer Rene – Fall To Pieces 1681 521
15. DJ Shah featuring Adrina Thorpe – Who Will Find Me (Original Summer Sunrise Mix) 1671 530
16. Albert Vorne – Formentera What (Gareth Emery Remix) 1663 479
17. Cerf, Mitiska & Jaren – Light The Skies (Retrobytes Classic Electrobounce Mix) 1554 488
18. Breakfast – The Sunlight 1534 473
19. Cosmic Gate – Body Of Conflict (Extended Vocal Mix) 1339 488
20. Above & Beyond featuring Zoe Johnston – Good For Me (Thomas Datt Bootleg Remix) 1328 410

Next week: ASOT YEARMIX 2007
"""

asot1119 = """
Star Wars – Imperial March (Alex M.O.R.P.H. Intro Edit
Luke Bond x M6 – Nexus [Armind] (Taken from A State of Trance 2023)
D72 – Our Darkness [Armada Captivating]
Armin van Buuren feat. Stuart Crichton – Dayglow (Bryan Kearney Remix) (Taken from A State of Trance 2023) [Armind]
Ahmed Helmy – Glitch (Taken from A State of Trance 2023) [Armind]
JODA – Breaking Down Walls (Myon’s Return to 2000 Mix) (Taken from A State of Trance 2023)  [Anjunabeats]
TUNE OF THE WEEK: Giuseppe Ottaviani & Ilan Bluestone – Futuro (Taken from A State of Trance 2023) [Armind] 
C-Systems – Atlantis (Taken from A State of Trance 2023) [A State Of Trance]
Andrea Ribeca – Space Line [Future Sound of Egypt]
FUTURE FAVORITE: Armin van Buuren & Justine Suissa – Burned with Desire (Bryan Kearney Remix) (Taken from A State of Trance 2023) [Armind] 
Solarstone – Solarcoaster (Maarten de Jong Remix)  (Taken from A State of Trance 2023) [Armada Music] 
Armin van Buuren – Computers Take Over The World (Maddix Remix) (Taken from A State of Trance 2023) [ [Armind]
South Of The Stars – Mechka [A State Of Trance]
Allen Watts & Bogdan Vix – Apache [Who’s Afraid Of 138!]
Paul Webster & NO-SO – Ignite [Who’s Afraid Of 138!]
SERVICE FOR DREAMERS: Rank 1 – Airwave [Free for All]
AVIRA feat. Dan Soleil – Wildfire [Armada Music]
RNX – Collective [Pure Trance]
Dennis Sheperd & Steve Brian – Summer Nights
Estiva – Carnal Emotion (Taken from Infinita) [Enhanced Colorized]
Ucast & Kamelon present CLAVIR – Radiate [Patterns]
Mark Roma & Jordan Grace – Wandering Heart [A State of Trance]
Scorz – Immersion [Armada Music] 
Steand – Stay [AVA Deep]
Daniel Wanrooy – Time [Flashover Recordings]
Kyau & Albert – Rising Hope [Euphonic]
Karen Overton – Your Loving Arms (Jose Amnesia Rework) [Armada Music]
Ferry Corsten – Connect [Flashover Recordings]
JODA – Breaking Down Walls (Myon’s Return to 2000 Mix) [Anjunabeats]"""

asots = [
    {
        "episode_id": "314",
        "text": asot314
    },
    {
        "episode_id": "325",
        "text": asot325
    },
    {
        "episode_id": "331",
        "text": asot331
    },
    {
        "episode_id": "119",
        "text": asot119
    }
]


In [ ]:
collection_name = "asot"
result = milvus_client.create_and_load_collection(collection_name, asots)

In [23]:
asot313 = """
[![A State of Trance](https://www.astateoftrance.com/wp-content/themes/asot/media/images/logo.svg)](https://www.astateoftrance.com/home/ "A State of Trance")

![A State of Trance](https://www.astateoftrance.com/wp-content/themes/asot/media/images/logo-word.svg)

MENU

![A State of Trance](https://www.astateoftrance.com/wp-content/themes/asot/media/images/logo-word.svg)

- [Instagram](http://instagram.com/asotlive "Instagram")
- [Facebook](https://www.facebook.com/astateoftrance "Facebook")
- [TikTok](https://www.tiktok.com/@asot.live "TikTok")
- [Spotify](https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce "Spotify")
- [Soundcloud](https://soundcloud.com/astateoftrance "Soundcloud")
- [YouTube](http://www.youtube.com/asotrecordings "YouTube")
- [Discord](https://discord.com/invite/arminvanbuuren "Discord")
- [Twitch](https://www.twitch.tv/asot "Twitch")
- [Newsletter](https://cm.to/A3bqwI6 "Newsletter")

Go back

# Episode 313

1\. Above & Beyond – Home (Wippenberg remix) (Anjunabeats)

2\. Heatbeat – Protoculture

3\. Envio – For You (The Blizzard remix)

4\. Talla 2XLC – No In Between (Duderstadt progressive remix)

5\. Paul van Dyk – La dolce Vita (Vandit)

6\. Paul Miller – Crystal Source (Conspiracy)

7\. FUTURE FAVORITE: John O Callagan feat. Audrey Gallagher – Big Sky (Agnelli & Nelson remix) (Armind)

8\. TUNE OF THE WEEK: The Doppler Effect – Beauty hides in the Deep (John O Callaghan remix)

9\. Thomas Bronzwaer – Resound (Sean Tyas remix) (United)

Hour 2: Live From Amnesia Ibiza 07-08-2007

1\. Nic Chagall – Back To San Fran (High Contrast)

2\. Black Pearl – Bounty Island (DJ Shah San Antonio Harbour remix)(ASOT)

3\. Purple Mood – One Night in Tokyo (DJ Shahs Savannah remix) (Anjunabeats)

4\. Ilse De Lange – The Great Escape (Armin van Buuren remix)

5\. Roland Klinkenberg – Mexico Can Wait (Global Underground)

6\. Unknown Artist – Unknown Title

7\. Genix – Phused (Marcus Schossow remix) (spinnin)

8\. Lange – Angel Falls (Signalrunners remix) (Lange Recordings)

9\. Dash Berlin – Till the Sky falls down (Dub mix) (Armada)

10\. Sultan & Ned – Together we Rise (Yoshitoshi)

#### Newsletter

A State of Trance Studio Visit

* * *"""